# MLflow × LLM ライブデモ（完全版）

このノートは、**Tracking → 比較 → アーティファクト → RetrieverをModel登録 → Prompt Registry** まで、
LLMアプリの実験管理に必要な最小構成をライブで見せるためのものです。

**所要時間目安**: 10–15分  
**前提**: 別ターミナルで `mlflow ui --port 5000` を起動しておくと、UIでRunがすぐ見られます。  
**UIのURL**: http://127.0.0.1:5000

---


## 1. セットアップ
**ねらい**: 余計なインフラなしで、すぐ実験管理を始められることを示す。

**UIで見るポイント**: `Experiments` に **mlflow-llm-live-demo** が作成され、Runが増える様子。


In [ ]:

# ▼ 会社・環境に合わせて必要ならコメントアウトを外してください
# !pip -q install mlflow pandas matplotlib scikit-learn

import os, json, time, random
import pandas as pd
import mlflow
from mlflow import MlflowClient

# ローカルファイルにトラッキング（サーバ不要）
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("mlflow-llm-live-demo")

print("Tracking URI:", mlflow.get_tracking_uri())
exp = mlflow.get_experiment_by_name("mlflow-llm-live-demo")
print("Experiment:", exp.name, "| ID:", exp.experiment_id)


## 2. 疑似LLM（ダミー）と評価セット
**ねらい**: APIキー不要で「プロンプト→出力→評価」の体験を再現。  
**MLflowの強み**: モデル種別に依存せず、**プロンプト/設定/出力**の記録フォーマットを統一できる。

In [ ]:

def dummy_llm(prompt: str, temperature: float = 0.2, max_tokens: int = 64):
    """非常に単純なダミー生成。温度で少し出力が揺れる。"""
    base_answers = {
        "2+2": "4",
        "3*5": "15",
        "capital of france": "Paris",
        "what is rag": "RAG is Retrieval-Augmented Generation."
    }
    p = prompt.lower()
    for k, v in base_answers.items():
        if k in p:
            if temperature > 0.5 and random.random() < min(temperature, 0.9):
                return v + " (approx)"
            return v
    return "I'm not sure." + (" Definitely." if temperature > 0.5 else "")

# 評価用の簡易データ
eval_set = pd.DataFrame([
    {"question": "2+2?", "answer": "4"},
    {"question": "3*5?", "answer": "15"},
    {"question": "What is RAG?", "answer": "RAG is Retrieval-Augmented Generation."},
    {"question": "Capital of France?", "answer": "Paris"}
])
eval_set


## 3. 1回の実験をMLflowに記録（params / metrics / artifacts）
**ねらい**: Run（1試行）単位で再現に必要な情報を完全保存できることを体感。  
**UIで見るポイント**: Run詳細 → **Parameters**, **Metrics**, **Artifacts**（predictions.json）。

In [ ]:

from statistics import mean

def exact_match(pred: str, ref: str) -> float:
    return 1.0 if pred.strip() == ref.strip() else 0.0

def run_experiment(model_name: str, temperature: float, max_tokens: int, seed: int = 42, **_ignore):
    random.seed(seed)
    rows, latencies, costs = [], [], []
    start = time.time()
    for _, row in eval_set.iterrows():
        q = row["question"]
        t0 = time.time()
        pred = dummy_llm(q, temperature=temperature, max_tokens=max_tokens)
        lat = time.time() - t0
        em = exact_match(pred, row["answer"])
        cost = max(1, len(pred)) * 0.001  # 仮の「トークンコスト」
        rows.append({"question": q, "prediction": pred, "reference": row["answer"], "em": em, "latency": lat, "cost": cost})
        latencies.append(lat); costs.append(cost)
    total = time.time() - start
    metrics = {
        "exact_match": mean([r["em"] for r in rows]),
        "latency_p50": pd.Series(latencies).quantile(0.5),
        "latency_p95": pd.Series(latencies).quantile(0.95),
        "total_time": total,
        "avg_cost": mean(costs),
    }
    return rows, metrics

with mlflow.start_run(run_name="single-run-demo") as run:
    params = {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.2, "max_tokens": 64}
    mlflow.log_params(params)

    results, metrics = run_experiment(**params)
    mlflow.log_metrics(metrics)

    # 予測詳細をArtifactsに保存
    os.makedirs("artifacts", exist_ok=True)
    with open("artifacts/predictions.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    mlflow.log_artifact("artifacts/predictions.json", artifact_path="eval")

    print("Run ID:", run.info.run_id)
    print("Metrics:", metrics)


## 4. ハイパラ違いで複数Runを記録 → 比較
**ねらい**: 条件差の効果を**定量比較**できる（感覚に頼らない）。  
**UIで見るポイント**: 複数Runを選択 → **Compare** → メトリクスの表・グラフ。

In [ ]:

search_space = [
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.1, "max_tokens": 64},
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.5, "max_tokens": 64},
    {"provider": "dummy", "model_name": "dummy-llm-v1", "temperature": 0.8, "max_tokens": 64},
]

for i, p in enumerate(search_space, 1):
    with mlflow.start_run(run_name=f"grid-run-{i}") as run:
        mlflow.log_params(p)
        results, metrics = run_experiment(**p)
        mlflow.log_metrics(metrics)
        with open(f"artifacts/preds_grid_{i}.json", "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        mlflow.log_artifact(f"artifacts/preds_grid_{i}.json", artifact_path="eval")
        print(f"Logged: {run.info.run_id}  -> EM={metrics['exact_match']:.2f}, p95={metrics['latency_p95']:.4f}, cost={metrics['avg_cost']:.4f}")


## 5. Notebookから上位Runを抽出（MlflowClient）
**ねらい**: UIだけでなく**プログラマブルに意思決定**できる。  
**UIで見るポイント**: `Experiments` → Run一覧のメトリクス表示と一致しているか。

In [ ]:

client = MlflowClient()
exp = client.get_experiment_by_name("mlflow-llm-live-demo")
runs = client.search_runs(
    experiment_ids=[exp.experiment_id],
    filter_string="",
    run_view_type=1,
    order_by=["metrics.exact_match DESC", "metrics.latency_p95 ASC"],
    max_results=10,
)
summary = []
for r in runs:
    summary.append({
        "run_id": r.info.run_id,
        "name": r.info.run_name,
        "exact_match": r.data.metrics.get("exact_match"),
        "latency_p95": r.data.metrics.get("latency_p95"),
        "avg_cost": r.data.metrics.get("avg_cost"),
        "temperature": r.data.params.get("temperature"),
        "model_name": r.data.params.get("model_name"),
    })
pd.DataFrame(summary)


## 6. アーティファクトで“失敗の中身”を見る
**ねらい**: スコアだけでなく、**どの質問で誤答したか**まで根拠を持って振り返れる。  
**UIで見るポイント**: Run詳細 → **Artifacts → eval → predictions.json** を開く。

In [ ]:

# 例として、直近のgrid-run-1のファイル名を推測して読み込み（ロバストでない簡易版）
# うまく見つからない場合は、UIからダウンロードしてNotebookに再アップロードしてください。
import glob
cand = sorted(glob.glob("mlruns/*/*/artifacts/eval/preds_grid_1.json"))
if cand:
    with open(cand[-1], "r", encoding="utf-8") as f:
        preds = json.load(f)
    pd.DataFrame(preds)
else:
    print("ローカルパス推測に失敗。UIのArtifactsから確認してください。")


## 7. UI 操作ガイド（ここを見せる）
1. **Experiments** → `mlflow-llm-live-demo` を開く  
2. Runを複数選択 → **Compare** で **EM/Latency/Cost** を比較  
3. 任意のRunをクリック → **Parameters / Metrics / Artifacts** を確認  
4. Artifactsの **eval/predictions.json** を開き、誤答ケースを確認  
5. ここまでで「**再現性・比較可能性・可観測性**」を実感してもらう

## 8. 実運用のヒント（口頭補足用）
- **RAG** では `embed_model`, `chunk_size`, `retriever_k`, `reranker` などを **params** に、  
  `retrieval_hit_rate`, `context_precision`, `hallucination_rate` などを **metrics** に。  
- **コスト監視**：投入/出力トークン数、P95レイテンシを必ずログ。  
- **シークレット**（APIキー）はparamsに入れない。環境変数・Secret管理を利用。

## 9. RetrieverをModelとして登録するデモ
**ねらい**: LLM本体はいじらなくても、**再現性が重要なコンポーネント**（検索器）をモデルとして管理できる。  
**UIで見るポイント**: Run詳細 → **Artifacts** に `corpus.json`、**Models** に登録済みモデル表示（バージョン管理）。

In [ ]:

# ▼ TF-IDFベースの簡単なRetrieverをモデル化
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import mlflow.pyfunc

corpus = [
    "Paris is the capital of France.",
    "RAG stands for Retrieval-Augmented Generation.",
    "2+2 equals 4.",
    "The capital of Japan is Tokyo."
]
doc_ids = [f"doc{i}" for i in range(len(corpus))]

class TfidfRetriever(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        with open(context.artifacts["corpus_path"], "r") as f:
            payload = json.load(f)
        self.doc_ids = payload["doc_ids"]
        self.vectorizer = TfidfVectorizer().fit(payload["corpus"])
        self.doc_mat = self.vectorizer.transform(payload["corpus"])
        self.top_k = int(context.model_config.get("top_k", 2))

    def predict(self, context, model_input):
        q_vec = self.vectorizer.transform(model_input["query"].tolist())
        sims = q_vec @ self.doc_mat.T
        topk_idx = np.argsort(-sims.toarray(), axis=1)[:, :self.top_k]
        results = []
        for idxs in topk_idx:
            results.append([self.doc_ids[i] for i in idxs])
        return pd.Series(results)

# アセット保存 → Artifactsとして一緒に記録
os.makedirs("assets", exist_ok=True)
with open("assets/corpus.json", "w") as f:
    json.dump({"corpus": corpus, "doc_ids": doc_ids}, f)

with mlflow.start_run(run_name="register-retriever-demo"):
    mlflow.pyfunc.log_model(
        artifact_path="retriever_model",
        python_model=TfidfRetriever(),
        artifacts={"corpus_path": "assets/corpus.json"},
        model_config={"top_k": 2},
        input_example=pd.DataFrame({"query": ["What is RAG?", "capital of France?"]})
    )

print("RetrieverをMLflowに登録しました。UIのArtifacts/Modelsから確認できます。")


## 10. Prompt Registryを使ったプロンプト管理デモ
**ねらい**: **プロンプト自体を資産として管理**（バージョン、エイリアス、ロールバック）。  
**UIで見るポイント**: `Prompts` タブ（※MLflowバージョン2.7+相当で利用可能）。  
**注意**: 環境のMLflowバージョンによりAPIが存在しない場合があります（下のセルは存在チェック付き）。

In [ ]:

prompt_text = "Answer the following question clearly: {{ question }}"

with mlflow.start_run(run_name="register-prompt-demo"):
    # バージョン互換性チェック（mlflow.prompts がない場合はスキップ）
    ok = hasattr(mlflow, "prompts") and hasattr(mlflow.prompts, "log_prompt")
    if ok:
        mlflow.prompts.log_prompt(
            "my_prompt_v1",
            template=prompt_text,
            input_variables=["question"]
        )
        print("✅ Prompt Registryに登録しました。UIの Prompts タブで確認できます。")
    else:
        print("ℹ️ お使いのMLflowでは Prompt Registry API が無効/未提供の可能性があります。")
        print("   MLflow 2.7+ 相当のドキュメントをご確認ください。")


## 11. まとめ（このデモで伝えたいこと）
- **再現性**：誰が・いつ・どの設定で・何が出たかを**Runに完全記録**  
- **比較可能性**：複数Runを**横並び**で比較し、品質/遅延/コストのトレードオフで意思決定  
- **可観測性**：Artifactsで**失敗の中身**まで追跡（必要に応じてTracingへ発展）  
- **資産化**：Retrieverなど**LLMを支える部品**をModelとして登録／PromptはRegistryで管理  
- 小さく始めて、必要に応じて **Model Registry / Prompt Registry / Tracing** に拡張可能